In [0]:
from pyspark.sql import SparkSession, Row
import re

# Crear sesión de Spark
spark = SparkSession.builder.appName("LogsNoEstructurados").getOrCreate()

# Leer el archivo como RDD
rdd_log = spark.sparkContext.textFile("/content/sample_data/accesos.log")

# Ver las primeras líneas
print("📄 Contenido del log:")
for linea in rdd_log.take(3):
    print(linea)

# Función para parsear una línea usando expresiones regulares
def parsear_log(linea):
    patron = r"\[(.*?)\] INFO - Usuario: (.*?) - Acción: (.*?) - IP: (.*)"
    match = re.match(patron, linea)
    if match:
        timestamp, usuario, accion, ip = match.groups()
        return Row(timestamp=timestamp, usuario=usuario, accion=accion, ip=ip)
    else:
        return None  # línea inválida

# Parsear cada línea y filtrar las que no coinciden
rdd_parseado = rdd_log.map(parsear_log).filter(lambda x: x is not None)

# Convertir a DataFrame
df_logs = spark.createDataFrame(rdd_parseado)

# Mostrar el resultado
df_logs.show(truncate=False)